In [1]:
import numpy as np

In [17]:
from scipy.io.arff import loadarff
from random import shuffle
lines,meta = loadarff('Ovarian.arff')
lines=lines.tolist()
#print lines[0]
shuffle(lines)
shuffle(lines)
shuffle(lines)
shuffle(lines)
shuffle(lines)
shuffle(lines)
#lines=[ i.split('\t') for i in lines]
categories=[i[len(i)-1] for i in lines]
lines=[i[0:len(i)-1] for i in lines]
lines=[[float(j) for j in i] for i in lines]

In [9]:
#from random import shuffle
#filename='braintumor.tab'
#with open(filename) as f:
#    lines=f.read().splitlines()
#    print len(lines)
#    lines=lines[3:]
#    shuffle(lines)
#    shuffle(lines)
#    shuffle(lines)
#    shuffle(lines)
#    shuffle(lines)
#    shuffle(lines)
#    lines=[ i.split('\t') for i in lines]
#    categories=[i[len(i)-1] for i in lines]
#    lines=[i[0:len(i)-1] for i in lines]
#    lines=[[float(j) for j in i] for i in lines]

43


In [18]:
print len(lines[0])
print len(lines)
print categories[0:10]


15154
253
['Normal', 'Normal', 'Cancer', 'Cancer', 'Normal', 'Cancer', 'Cancer', 'Cancer', 'Normal', 'Normal']


In [20]:
for i in range(len(lines[0])):
    base=0
    for j in range(len(lines)):
        if(base>lines[j][i]):
            base=lines[j][i]
    base=base*-1
    for j in range(len(lines)):
        lines[j][i]+=base
for i in range(len(lines[0])):
    base=1
    for j in range(len(lines)):
        if(base<lines[j][i]):
            base=lines[j][i]
    for j in range(len(lines)):
        lines[j][i]/=base

In [21]:
lines=lines[228:253]+lines[0:228]
categories=categories[228:253]+categories[0:228]

In [22]:
C=sorted(list(set(categories)))
Map= dict((c, i) for i, c in enumerate(C))
Y=[]
for i in categories:
    Y.append(Map[i])
print Y[0:20]

[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0]


In [23]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,f_classif, mutual_info_classif
f=SelectKBest(chi2, k=2000)
xlines = f.fit_transform(lines[0:228], Y[0:228])
mast=f.get_support()

In [24]:
feachers=[]
i=0
for bool in mast:
    if bool:
        feachers.append(i)
    i+=1

In [25]:
from keras.utils import np_utils
Y=np_utils.to_categorical(Y)
X=np.array(xlines)
print(len(xlines[0]))

Using TensorFlow backend.


2000


In [26]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout ,ActivityRegularization,LeakyReLU


In [29]:
model = Sequential()
model.add(Dense(3000, activation='linear',input_shape=(2000,)))
model.add(LeakyReLU(alpha=.001))
model.add(ActivityRegularization(l2=0.001))
model.add(Dense(1000, activation='linear'))
model.add(LeakyReLU(alpha=.001))
model.add(ActivityRegularization(l2=0.001))
#model.add(Dense(800, activation='elu'))
#model.add(ActivityRegularization(l2=0.01))
#model.add(Dense(500, activation='elu'))
#model.add(Dense(100, activation='elu'))
#model.add(Dense(100, activation='elu'))
#model.add(Dense(100, activation='elu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 3000)              6003000   
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 3000)              0         
_________________________________________________________________
activity_regularization_3 (A (None, 3000)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1000)              3001000   
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 1000)              0         
_________________________________________________________________
activity_regularization_4 (A (None, 1000)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 2002      
Total para

In [30]:
model.fit(X,Y[0:228],epochs=20)


Epoch 1/20
228/228 [==============================] - 2s - loss: 2.9346 - acc: 0.7237     
Epoch 2/20
228/228 [==============================] - 2s - loss: 0.5480 - acc: 0.9298     
Epoch 3/20
228/228 [==============================] - 2s - loss: 0.3804 - acc: 0.9649     
Epoch 4/20
228/228 [==============================] - 2s - loss: 0.3076 - acc: 0.9781     
Epoch 5/20
228/228 [==============================] - 2s - loss: 0.2502 - acc: 0.9956     
Epoch 6/20
228/228 [==============================] - 2s - loss: 0.2409 - acc: 1.0000     
Epoch 7/20
228/228 [==============================] - 2s - loss: 0.2143 - acc: 0.9781     
Epoch 8/20
228/228 [==============================] - 2s - loss: 0.1711 - acc: 1.0000     
Epoch 9/20
228/228 [==============================] - 2s - loss: 0.1545 - acc: 1.0000     
Epoch 10/20
228/228 [==============================] - 2s - loss: 0.1655 - acc: 0.9956     
Epoch 11/20
228/228 [==============================] - 2s - loss: 0.1355 - acc: 1.0000   

In [31]:
testx=[]
for j in range(228,253,1):
    m=[lines[j][i] for i in feachers]
    testx.append(m)
        
testx=np.array(testx)

score = model.evaluate(testx,Y[228:] ,verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

('Test loss:', 0.10078590363264084)
('Test accuracy:', 1.0)
